In [ ]:
import numpy as np 
import flexring as flx 
from euclid3 import Vector2
import matplotlib.pyplot as plt
def interpolate_boundary_condition(centre:Vector2,
                                   radius:float,
                                   point1:Vector2,
                                   point2:Vector2,
                                   dydx:float,
                                   ddydx:float,
                                   beta:float,
                                   direction:int):
    # given a chord or a semi chord on a circle, finds the 
    # point along the chord where separation happens
    dr_1 = (point1 - centre).magnitude() - radius
    dr_2 = (point2 - centre).magnitude() - radius
    dr_dtheta_1 = flx.polar_derivative(point= point1 - centre,dy=dydx)
    dr_dtheta_2 = flx.polar_derivative(point= point2- centre,dy= dydx)
    ddr_dtheta_1 = flx.polar_second_derivative(point1 - centre,dydx,ddydx)
    ddr_dtheta_2 = flx.polar_second_derivative(point2 - centre, dydx,ddydx)                                           
    print(f"r1: {dr_1}\nr2: {dr_2}\ndr1:{dr_dtheta_1}\ndr2: {dr_dtheta_1}\nddr1: {ddr_dtheta_1}\nddr2: {ddr_dtheta_2}")
    
    # condition is true when lhs > rhs
    # we calculate lhs and rhs at point1 and point2 and linearly interpolate
    # to find the point where they cross
    lhs_1 = 0.5*ddr_dtheta_1
    lhs_2 = 0.5*ddr_dtheta_2
    rhs_1 = -2*(beta**2)*(direction*dr_dtheta_1/beta + dr_1)
    rhs_2 = -2*(beta**2)*(direction*dr_dtheta_2/beta + dr_2)
    t = (rhs_1 - lhs_1)/((lhs_2-lhs_1)-(rhs_2 - rhs_1))# where rhs == lhs
    return point1 + t*(point2 - point1), t

beta = 3
t = np.linspace(0 , 1 , 100)
radius = 0.35
distance_to_edge = np.linspace(radius , radius-0.1, 100)
angles = np.deg2rad(np.linspace(170, 10, 160))
tyre_centres = [[Vector2(r*np.cos(theta), r*np.sin(theta))for r in distance_to_edge] for theta in angles]
p2_angle = -np.arccos((radius - penetration)/(radius))
p2 = radius * Vector2(np.cos(p2_angle), np.sin(p2_angle))
p1 = Vector2(tyre_centre.x , p2.y)
points = [p1 + T*(p2 - p1) for T in t]
lhs = [0.5*flx.polar_second_derivative(p, 0 , 0) for p in points]
get_rhs = lambda P : -2*(beta**2)*(flx.polar_derivative(P , 0)/beta + P.magnitude()-radius)
rhs = [get_rhs(p) for p in points]
crossing_point, T = flx.interpolate_boundary_condition(point1=p1,
                                                    point2=p2,
                                                    centre = tyre_centre,
                                                    radius = radius,
                                                    dydx = 0,
                                                    ddydx = 0,
                                                    beta = beta,
                                                    direction=1)
###### 
plt.plot(t , lhs)
plt.plot(t , rhs)
plt.plot([0,1], [rhs[0], rhs[-1]])
plt.plot(T , rhs[0] + T*(rhs[-1] - rhs[0]), "*")
plt.show()

In [ ]:
import flexring as flx
import matplotlib.pyplot as plt
import numpy as np
%matplotlib widget
road = flx.Road(
                step_width=0.01,
                step_height=0.08,
                step_profile_phase=np.pi
                )
fig,ax =plt.subplots(3 , 1, sharex=True)
ax[0].plot(road.x , road.y , marker=".")
ax[1].plot(road.x, road.dydx, marker=".")
ax[2].plot(road.x , road.ddydx, marker=".")
plt.xlim((2.49, 2.51))
plt.show()
